<a href="https://colab.research.google.com/github/G544/galinanana_ml/blob/main/Recomendational_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"galinanana","key":"cbb73354e6a4418bcb42b7b2ca579b64"}'}

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"galinanana","key":"cbb73354e6a4418bcb42b7b2ca579b64"}
import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d prajitdatta/movielens-100k-dataset
! git clone https://github.com/G544/galinanana_ml.git

  0% 0.00/4.77M [00:00<?, ?B/s]
100% 4.77M/4.77M [00:00<00:00, 117MB/s]
Cloning into 'galinanana_ml'...
remote: Enumerating objects: 371, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 371 (delta 53), reused 14 (delta 4), pack-reused 240
Receiving objects: 100% (371/371), 21.93 MiB | 646.00 KiB/s, done.
Resolving deltas: 100% (92/92), done.


In [4]:
import io
import pandas as pd
import zipfile
from pathlib import Path
from sklearn.model_selection import train_test_split

In [5]:
zip = zipfile.ZipFile('/content/movielens-100k-dataset.zip')
names = (zip.namelist())

In [6]:
names
names_table = ['u.data', 'u.genre', 'u.info', 'u.item', 'u.occupation', 'u.user']
cols_data = []

In [7]:
data_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
user_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
item_cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 
             'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 
             'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
info_cols = ['amount', 'category']

u_data = pd.read_csv(io.BytesIO(zip.read(names[3])), sep='\t', encoding='latin-1', names = data_cols)
genre_list = pd.read_csv(io.BytesIO(zip.read(names[4])), sep='|', encoding='latin-1')['unknown']
u_info = pd.read_csv(io.BytesIO(zip.read(names[5])), sep='|', encoding='latin-1', names = info_cols)
u_item = pd.read_csv(io.BytesIO(zip.read(names[6])), sep='|', encoding='latin-1', names = item_cols)
occupation_list = pd.read_csv(io.BytesIO(zip.read(names[7])), sep='|', encoding='latin-1', names = [1])[1]
u_user = pd.read_csv(io.BytesIO(zip.read(names[8])), sep='|', encoding='latin-1', names = user_cols)

# Создание БД


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sqlite3

In [ ]:
path = "./drive/My Drive/databases/mydatabase.db"

In [ ]:
conn = sqlite3.connect(path)


In [ ]:
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()
# Создание простейшей таблицы, все поля (столбцы) которой имеют тип text
# Создание таблицы u_data 
'''u.data -- The full u data set, 100000 ratings by 943 users on 1682 items.
Each user has rated at least 20 movies. Users and items are
numbered consecutively from 1. The data is randomly
ordered. This is a tab separated list of
user id | item id | rating | timestamp.'''

cursor.execute("CREATE TABLE data (user_id, movie_id, rating, unix_timestamp)")
sql  = "INSERT INTO data VALUES (?, ?, ?, ?)"
for i in range(u_data.shape[0]):
  val = u_data.loc[i].to_list()
  cursor.execute(sql, val)
conn.commit()
# Создание таблицы u_user
'''
u.user -- Demographic information about the users; this is a tab
separated list of
user id | age | gender | occupation | zip code
The user ids are the ones used in the u.data data set.'''

cursor.execute("CREATE TABLE user ('user id','age','gender','occupation','zip code')")
sql  = "INSERT INTO user VALUES (?, ?, ?, ?, ?)"
for i in range(u_user.shape[0]):
  val = [str(s) for s in u_user.loc[i].to_list()]
  cursor.execute(sql, val)
conn.commit()
# Создание таблицы u_item
'''
u.item -- Information about the items (movies); this is a tab separated
list of
movie id | movie title | release date | video release date |
IMDb URL | unknown | Action | Adventure | Animation |
Children's | Comedy | Crime | Documentary | Drama | Fantasy |
Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
Thriller | War | Western |'''

cursor.execute("CREATE TABLE items (movie_id, movie_title, release_date, video_release_date, IMDb_URL, unknown, Action, Adventure,Animation, Children, Comedy, Crime, Documentary, Drama, Fantasy, FilmNoir, Horror, Musical, Mystery, Romance, Sci_Fi, Thriller, War, Western)")
sql  = "INSERT INTO items VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
for i in range(u_item.shape[0]):
  val = [str(s) for s in u_item.loc[i].to_list()]
  cursor.execute(sql, val)
conn.commit()

# Закрываем курсор
cursor.close()
# Закрываем соединение (рекомендуется)
conn.close()

# Feature extractor


In [8]:
df = u_data.merge(u_user, how='left', on = 'user_id')
df = df.merge(u_item, how='left', on='movie_id')

In [9]:
drop_columns = ['movie_title','video_release_date', 'IMDb_URL', 'zip_code', 'movie_id', 'user_id', 'unknown']

In [10]:
data1 = df.drop(drop_columns,axis=1)

In [11]:
target_col = ['rating']
cat = ['sex', 'occupation'] 
cat_proc = ['Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 
       'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 
       'Sci-Fi', 'Thriller', 'War', 'Western']

real = ['age','unix_timestamp']
date = ['release_date']

In [12]:
data1 = pd.concat([data1,pd.get_dummies(data1[cat])], axis=1)
data1.drop(cat,axis=1, inplace=True)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data1[real] = scaler.fit_transform(data1[real])

In [14]:
import datetime
data1['release_date'] = pd.to_datetime(data1['release_date'])
data1['release_date_year'] = [s.year for s in data1['release_date']]
data1['release_date_year'] = [s.month for s in data1['release_date']]
data1['release_date_year'] = [s.day for s in data1['release_date']]
data1.drop('release_date', axis=1, inplace=True)

# Baseline

In [15]:
from sklearn.model_selection import train_test_split
target = data1[target_col]
X = data1.drop(target_col, axis=1).fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X, target)


In [16]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [25]:
from sklearn.metrics import auc, accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score, mean_squared_error
import numpy as np
test_pred = nb.predict(X_test)
train_pred = nb.predict(X_train)
print(accuracy_score(y_test, test_pred))
print(precision_score(y_test, test_pred, average = 'weighted'))
print(recall_score(y_test, test_pred, average = 'weighted'))


0.27496
0.3020136642888187
0.27496


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

<ipython-input-127-2ae5f042fe42>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestClassifier()

In [ ]:
test_pred = rf.predict(X_test)
train_pred = rf.predict(X_train)
print(accuracy_score(y_test, test_pred))
print(precision_score(y_test, test_pred, average = 'weighted'))
print(recall_score(y_test, test_pred, average = 'weighted'))

0.35904
0.35368453289488794
0.35904


Neural Colaborative Filter

In [42]:
import torch
from torch.utils.data import DataLoader
from pt_layer import FeaturesEmbedding, MultiLayerPerceptron
import tqdm

ratings=u_data

class MovieLensDataset(torch.utils.data.Dataset):
    """
    MovieLens Dataset
    Data preparation
        treat samples with a rating less than 3 as negative samples
    """
    def __init__(self, ratings):
        data = ratings.copy().to_numpy()
        self.items = data[:, :2].astype(np.int32) - 1  # -1 because ID begins from 1
        self.targets = self.__preprocess_target(data[:, 2]).astype(np.float32)
        self.field_dims = np.max(self.items, axis=0) + 1
        self.user_field_idx = np.array((0, ), dtype=np.int64)
        self.item_field_idx = np.array((1,), dtype=np.int64)

    def __len__(self):
        return self.targets.shape[0]

    def __getitem__(self, index):
        return self.items[index], self.targets[index]

    def __preprocess_target(self, target):
        target[target < 3] = 0
        target[target >= 3] = 1
        return target

In [37]:
def get_dataset():
    return MovieLensDataset(ratings)

In [29]:
class NeuralCollaborativeFiltering(torch.nn.Module):
    """
    A pytorch implementation of Neural Collaborative Filtering.
    Reference:
        X He, et al. Neural Collaborative Filtering, 2017.
    """

    def __init__(self, field_dims, user_field_idx, item_field_idx, embed_dim, mlp_dims, dropout):
        super().__init__()
        self.user_field_idx = user_field_idx
        self.item_field_idx = item_field_idx
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.embed_output_dim = len(field_dims) * embed_dim
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout, output_layer=False)
        self.fc = torch.nn.Linear(mlp_dims[-1] + embed_dim, 1)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_user_fields)``
        """
        x = self.embedding(x)
        user_x = x[:, self.user_field_idx].squeeze(1)
        item_x = x[:, self.item_field_idx].squeeze(1)
        x = self.mlp(x.view(-1, self.embed_output_dim))
        gmf = user_x * item_x
        x = torch.cat([gmf, x], dim=1)
        x = self.fc(x).squeeze(1)
        return torch.sigmoid(x)

In [30]:
class EarlyStopper(object):

    def __init__(self, num_trials, save_path):
        self.num_trials = num_trials
        self.trial_counter = 0
        self.best_accuracy = 0
        self.save_path = save_path

    def is_continuable(self, model, accuracy):
        if accuracy > self.best_accuracy:
            self.best_accuracy = accuracy
            self.trial_counter = 0
            torch.save(model, self.save_path)
            return True
        elif self.trial_counter + 1 < self.num_trials:
            self.trial_counter += 1
            return True
        else:
            return False

In [31]:
def get_model(dataset):
    field_dims = dataset.field_dims
    return NeuralCollaborativeFiltering(field_dims, embed_dim=64, mlp_dims=(32, 32), dropout=0.2,
                                       user_field_idx=dataset.user_field_idx,
                                       item_field_idx=dataset.item_field_idx) 

In [32]:
def train(model, optimizer, data_loader, criterion, device, log_interval=100):
    model.train()
    total_loss = 0
    tk0 = tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)
    for i, (fields, target) in enumerate(tk0):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            tk0.set_postfix(loss=total_loss / log_interval)
            total_loss = 0

In [33]:
def test(model, data_loader, device):
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for fields, target in tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0):
            fields, target = fields.to(device), target.to(device)
            y = model(fields)
            targets.extend(target.tolist())
            predicts.extend(y.tolist())
    return roc_auc_score(targets, predicts)

In [34]:
device_cpu = "cpu"
learning_rate = 0.001
weight_decay = 1e-6
batch_size = 32
epochs = 50
model_name = "ncf"
device = torch.device(device_cpu)

In [38]:
dataset = get_dataset()
train_length = int(len(dataset) * 0.8)
valid_length = int(len(dataset) * 0.1)
test_length = len(dataset) - train_length - valid_length

train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
        dataset, (train_length, valid_length, test_length))

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=4)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=4)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [43]:
model = get_model(dataset).to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=weight_decay)
early_stopper = EarlyStopper(num_trials=5, save_path=f'{model_name}.pt')

In [48]:
for epoch_i in range(epochs):
    train(model, optimizer, train_data_loader, criterion, device)
    auc = test(model, valid_data_loader, device)
    print('epoch:', epoch_i, 'validation: auc:', auc)
    if not early_stopper.is_continuable(model, auc):
        print(f'validation: best auc: {early_stopper.best_accuracy}')
        break
    auc = test(model, test_data_loader, device)
    print(f'test auc: {auc}')

  0%|          | 0/2500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 313/313 [00:00<00:00, 360.27it/s]


epoch: 0 validation: auc: 0.7778974482732667


  0%|          | 0/313 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 313/313 [00:00<00:00, 364.03it/s]


test auc: 0.786476349094154


  0%|          | 0/2500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 313/313 [00:01<00:00, 276.69it/s]


...

epoch: 4 validation: auc: 0.7796955110295627


  0%|          | 0/313 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 313/313 [00:01<00:00, 260.97it/s]


test auc: 0.7753730205071645


  0%|          | 0/2500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
 25%|██▌       | 630/2500 [00:03<00:10, 171.45it/s, loss=0.295]


KeyboardInterrupt: ignored